In [5]:
import redis

In [6]:
rClient = redis.Redis(host='localhost', port=6379, db=0)

In [11]:
def cache(expiry=100):
    def _get_cache(f):
        def get(self, key):
            #prepending the key with function name so that different functions don't 
            #overwrite the same key
            key_to_check = '{}:{}'.format( f.__name__, key )
            value = self.getFromCache(key_to_check)
            if value:
                return value
            value = f(self, key)
            self.setInCache(key_to_check, value, expiry)
            return value
        return get
    return _get_cache

In [65]:
class API:
    def __init__(self, r_client):
        self.r_client = r_client
        self.expiry = 300

    def getFromCache(self, key):
        return self.r_client.get(key)
    
    def setInCache(self, key, value, expiry = None):
        self.r_client.set(key, value, expiry or self.expiry)
    
    @cache(expiry=500)
    def _request(self, key):
        import time
        time.sleep(3)
        return key**2
    
    def request(self, key):
        return float( self._request(key) )
    
    def delete(self, key = None):
        if not key:
            self.r_client.flushall()
            return
        keys = self.r_client.keys(f'*{key}*')
        key = keys[0] if keys else key
        self.r_client.delete(key)

In [66]:
api = API(r_client=rClient)

In [71]:
api.request(17)

289.0

In [73]:
api.delete(17)